# Data cleaning

Data cleaning of credit_card_balance.csv and installments_payments.csv.

In [4]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler

## Credit card balance

- Monthly balance snapshots of previous credit cards that the applicant has with Home Credit.

- This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credit cards * # of months where we have some history observable for the previous credit card) rows.

CUSTOMER RISK PROFILE

- Number of Loans per Customer
- Rate at which Loan is paid back by customer - No of instalments per customer per loan
- How much did the Customer load a Credit line?
- How many times did the Customer miss the minimum payment?
- What is the average number of days did Customer go past due date?
- What fraction of minimum payments were missed?

CUSTOMER BEHAVIOUR PATTERNS

- Cash withdrawals VS Overall Spending ratio
- Average number of drawings per customer - Total Drawings / Number of Drawings

In [5]:
# Load data

credit_card_balance = pd.read_csv('credit_card_balance.csv', sep=',')

credit_card_balance.head(10)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.500,0.0,877.500,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.000,0.0,0.000,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.000,0.0,0.000,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.000,0.0,0.000,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.000,0.0,11547.000,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0
5,2646502,380010,-7,82903.815,270000,0.0,0.000,0.0,0.000,4449.105,...,82773.315,82773.315,0.0,0,0.0,0.0,2.0,Active,7,0
6,1079071,171320,-6,353451.645,585000,67500.0,67500.000,0.0,0.000,14684.175,...,351881.145,351881.145,1.0,1,0.0,0.0,6.0,Active,0,0
7,2095912,118650,-7,47962.125,45000,45000.0,45000.000,0.0,0.000,0.000,...,47962.125,47962.125,1.0,1,0.0,0.0,51.0,Active,0,0
8,2181852,367360,-4,291543.075,292500,90000.0,289339.425,0.0,199339.425,130.500,...,286831.575,286831.575,3.0,8,0.0,5.0,3.0,Active,0,0
9,1235299,203885,-5,201261.195,225000,76500.0,111026.700,0.0,34526.700,6338.340,...,197224.695,197224.695,3.0,9,0.0,6.0,38.0,Active,0,0


In [6]:
# Identify duplicate rows

duplicate_rows = credit_card_balance[credit_card_balance.duplicated()]
len(duplicate_rows)

0

### Feature selection

In [7]:
# Feature selection

credit_card_balance.loc[0:10, ["SK_ID_PREV", "SK_ID_CURR", "AMT_PAYMENT_CURRENT", "AMT_PAYMENT_TOTAL_CURRENT"]]

,SK_ID_PREV,SK_ID_CURR,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT
0,2562384,378907,1800.000,1800.000
1,2582071,363914,2250.000,2250.000
2,1740877,371185,2250.000,2250.000
3,1389973,337855,11925.000,11925.000
4,1891521,126868,27000.000,27000.000
5,2646502,380010,3825.000,3825.000
6,1079071,171320,15750.000,15750.000
7,2095912,118650,264.690,0.000
8,2181852,367360,4093.515,4093.515
9,1235299,203885,45000.000,45000.000


In [8]:
credit_card_balance.loc[credit_card_balance["AMT_PAYMENT_CURRENT"] < credit_card_balance["AMT_PAYMENT_TOTAL_CURRENT"],
                        :].loc[:, ["AMT_PAYMENT_CURRENT", "AMT_PAYMENT_TOTAL_CURRENT"]]

,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT


AMT_PAYMENT_CURRENT is never lower than AMT_PAYMENT_TOTAL_CURRENT (it is always higher or equal, or they are both null), which means that AMT_PAYMENT_CURRENT contains the value in AMT_PAYMENT_TOTAL_CURRENT. Therefore, we can drop that column.

In [9]:
credit_card_balance.loc[credit_card_balance["AMT_DRAWINGS_CURRENT"] < credit_card_balance["AMT_DRAWINGS_ATM_CURRENT"], :].loc[:, ["AMT_DRAWINGS_CURRENT",
                                                                                                                                  "AMT_DRAWINGS_ATM_CURRENT"]]

,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_ATM_CURRENT
438776,-1687.50,0.0
747302,-519.57,0.0
3284667,-6211.62,0.0


Same thing for AMT_DRAWINGS_CURRENT and AMT_DRAWINGS_ATM_CURRENT. Even though the amount of drawings in the ATM may be zero, the amount of drawings in total can be negative - when the amount of money drawing during a month by a client is lower than zero, it means that the client has made a payment to the bank that is greater than the outstanding balance on their credit card.

Therefore, we can drop AMT_DRAWINGS_ATM_CURRENT, because AMT_DRAWINGS_CURRENT contains all the information. Same thing applies to AMT_DRAWINGS_POS_CURRENT. 

In [10]:
credit_card_balance.loc[0:10, ["AMT_DRAWINGS_CURRENT", "AMT_DRAWINGS_OTHER_CURRENT"]]

,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT
0,877.500,0.0
1,2250.000,0.0
2,0.000,0.0
3,2250.000,0.0
4,11547.000,0.0
5,0.000,0.0
6,67500.000,0.0
7,45000.000,0.0
8,289339.425,0.0
9,111026.700,0.0


In [11]:
credit_card_balance.loc[credit_card_balance["AMT_DRAWINGS_CURRENT"] < credit_card_balance["AMT_DRAWINGS_OTHER_CURRENT"], :].loc[:, ["AMT_DRAWINGS_CURRENT", "AMT_DRAWINGS_OTHER_CURRENT"]]

,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT
438776,-1687.50,0.0
747302,-519.57,0.0
3284667,-6211.62,0.0


We can also drop this column.

Regarding AMT_RECEIVABLE_PRINCIPAL, AMT_RECIVABLE and AMT_TOTAL_RECEIVABLE, since AMT_TOTAL_RECEIVABLE is the total amount receivable on the previous credit, that contains the number we need and we can drop the other two columns.

Considering the columns we decided to drop, regarding the 4 columns with the number of drawings in the month in matter, it only makes sense to keep CNT_DRAWINGS_CURRENT, because that's what we can connect to the information that wasn't dropped. Also, that column contains total number of drawings, while the other 3 contained very specific drawings.

In [12]:
# Drop the columns

credit_card_balance.drop(['AMT_PAYMENT_TOTAL_CURRENT', 'AMT_DRAWINGS_ATM_CURRENT',
                          'AMT_DRAWINGS_POS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT',
                          "AMT_RECEIVABLE_PRINCIPAL", "AMT_RECIVABLE",
                          "CNT_DRAWINGS_ATM_CURRENT", "CNT_DRAWINGS_OTHER_CURRENT",
                          "CNT_DRAWINGS_POS_CURRENT", "AMT_INST_MIN_REGULARITY"],
                         axis=1, inplace=True)

In [13]:
credit_card_balance

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_CURRENT,AMT_PAYMENT_CURRENT,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,877.5,1800.00,0.000,1,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.00,64875.555,1,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,2250.00,31460.085,0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,11925.00,233048.970,1,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,11547.0,27000.00,453919.455,1,101.0,Active,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3840307,1036507,328243,-9,0.000,45000,0.0,NaN,0.000,0,0.0,Active,0,0
3840308,1714892,347207,-9,0.000,45000,0.0,1879.11,0.000,0,23.0,Active,0,0
3840309,1302323,215757,-9,275784.975,585000,270000.0,375750.00,273093.975,2,18.0,Active,0,0
3840310,1624872,430337,-10,0.000,450000,0.0,NaN,0.000,0,0.0,Active,0,0


### Dealing with missing values

In [14]:
# Deal with missing values

n_null_dict = {}

for column in credit_card_balance.columns:
    null_sum = credit_card_balance.isnull()[column].sum()
    n_null_dict[column] = null_sum

n_null = pd.DataFrame(n_null_dict, index = ["Number of missing values"])

In [15]:
n_null

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_CURRENT,AMT_PAYMENT_CURRENT,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
Number of missing values,0,0,0,0,0,0,767988,0,0,305236,0,0,0


We could replace the missing values using a statistical method such as mean, median, or mode. However, each client is different and has different ATM movements, so imputing with a single value may not be appropriate.

Therefore, the approach we will take is to impute the missing values based on similar clients: group clients by some relevant characteristics.

AMT_DRAWINGS_CURRENT seems to be the variable with the highest correlation with AMT_PAYMENT_CURRENT. Let's group clients based on that.

In [16]:
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, app_train, y=None):
        return self
    def transform(self, app_train):
        return app_train[self.attribute_names].values
    def get_feature_names(self):
        return self.attribute_names

In [17]:
# Function that replaces missing values to be used in Imputer

class ReplaceNull(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, credit_card_balance, y=None):
        return self
    def transform(self, credit_card_balance):
        
        corr_matrix = credit_card_balance.corr().abs()
        pearson_var = corr_matrix[self.attribute_names]
        most_correlated = pearson_var.sort_values(ascending=False).index[1]
    
        # Create new column with quantile bin for most_correlated variable
    
        credit_card_balance['quantile'] = pd.qcut(credit_card_balance[most_correlated], 2,
                                                  labels=False, duplicates='drop')

        # Compute median for each quantile
    
        quantile_medians = credit_card_balance.groupby('quantile')[most_correlated].median()

        # Replace missing values in the chosen column with the median for the corresponding quantile
    
        for quantile in range(2):
            mask = (credit_card_balance['quantile'] == quantile) & credit_card_balance[self.attribute_names].isna()
            df.loc[mask, self.attribute_names] = quantile_medians[quantile]
    
        # Remove the temporary quantile column
    
        credit_card_balance.drop('quantile', axis=1, inplace=True)
            
        return credit_card_balance[self.attribute_names]
    
"""
def quartiles(df, column_name):
    
    # Compute correlation matrix and get most correlated variable
    
    corr_matrix = df.corr().abs()
    most_correlated = corr_matrix[column_name].sort_values(ascending=False).index[1]
    
    # Create new column with quantile bin for most_correlated variable
    
    df['quantile'] = pd.qcut(df[most_correlated], 2, labels=False, duplicates='drop')

    # Compute median for each quantile
    
    quantile_medians = df.groupby('quantile')[most_correlated].median()

    # Replace missing values in the chosen column with the median for the corresponding quantile
    
    for quantile in range(2):
        mask = (df['quantile'] == quantile) & df[column_name].isna()
        df.loc[mask, column_name] = quantile_medians[quantile]
    
    # Remove the temporary quantile column
    
    df.drop('quantile', axis=1, inplace=True)
        
    return df
"""

"\ndef quartiles(df, column_name):\n    \n    # Compute correlation matrix and get most correlated variable\n    \n    corr_matrix = df.corr().abs()\n    most_correlated = corr_matrix[column_name].sort_values(ascending=False).index[1]\n    \n    # Create new column with quantile bin for most_correlated variable\n    \n    df['quantile'] = pd.qcut(df[most_correlated], 2, labels=False, duplicates='drop')\n\n    # Compute median for each quantile\n    \n    quantile_medians = df.groupby('quantile')[most_correlated].median()\n\n    # Replace missing values in the chosen column with the median for the corresponding quantile\n    \n    for quantile in range(2):\n        mask = (df['quantile'] == quantile) & df[column_name].isna()\n        df.loc[mask, column_name] = quantile_medians[quantile]\n    \n    # Remove the temporary quantile column\n    \n    df.drop('quantile', axis=1, inplace=True)\n        \n    return df\n"

### Final pipeline to transform data and get new dataframe:

In [18]:
num_variables = [col for col in credit_card_balance.columns if col != 'NAME_CONTRACT_STATUS' and col != 'SK_ID_PREV' and col != 'SK_ID_CURR' and col != 'AMT_PAYMENT_CURRENT' and col != 'CNT_INSTALMENT_MATURE_CUM']

cat_variables = ["NAME_CONTRACT_STATUS"]

miss_values_variables = ["AMT_PAYMENT_CURRENT", "CNT_INSTALMENT_MATURE_CUM"]

In [19]:
num_pipeline = Pipeline([
        ('selector', DataFrameSelector(num_variables)),
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler())
        ])
cat_pipeline = Pipeline([
        ('selector', DataFrameSelector(cat_variables)),
        ('imputer', SimpleImputer(strategy="most_frequent")),
        ('cat_encoder', OneHotEncoder())
        ])
miss_values_pipeline = Pipeline([
        ('replace_null', ReplaceNull(miss_values_variables)),
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler())
        ])

In [20]:
preprocess_pipeline = ColumnTransformer([
        ("num_pipeline", num_pipeline, num_variables),
        ("cat_pipeline", cat_pipeline, cat_variables),
        ("miss_values_pipeline", miss_values_pipeline, miss_values_variables)
])

In [21]:
preprocess_pipeline.fit(credit_card_balance)

preprocessed_credit_card_balance = preprocess_pipeline.transform(credit_card_balance)

TypeError: sort_values() missing 1 required positional argument: 'by'

In [50]:
# Pipeline to transform numerical and categorical features

numerical_features = [col for col in credit_card_balance.columns if col != 'NAME_CONTRACT_STATUS' and col != 'SK_ID_PREV' and col != 'SK_ID_CURR']

categorical_features = ["NAME_CONTRACT_STATUS"]

# Define column transformer to apply scaling and one-hot encoding

preprocessor = make_column_transformer(
    (MinMaxScaler(), numerical_features),
    (OneHotEncoder(), categorical_features)
)

# Define the pipeline to apply the column transformer

pipeline = make_pipeline(preprocessor)

# Fit the pipeline to the original dataset 

pipeline.fit(credit_card_balance)

# Get the transformed numerical feature names

numerical_feature_names = numerical_features

# Get the transformed categorical feature names

categorical_feature_names = pipeline.named_steps['columntransformer'].transformers_[1][1].get_feature_names(categorical_features)

# Combine the feature names

feature_names = numerical_feature_names + categorical_feature_names.tolist()

# Select the columns to keep from the original dataframe

id_columns = ['SK_ID_PREV', 'SK_ID_CURR']

# Create a new dataframe with the transformed features

new_credit_card_balance = pd.DataFrame(pipeline.transform(credit_card_balance), columns=feature_names)

# Concatenate the transformed features dataframe with the original id columns

credit_card_balance = pd.concat([credit_card_balance[id_columns], new_credit_card_balance], axis=1)

/Users/patriciafonseca/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [51]:
credit_card_balance

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_CURRENT,AMT_PAYMENT_CURRENT,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_CURRENT,CNT_INSTALMENT_MATURE_CUM,SK_DPD,SK_DPD_DEF,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Sent proposal,NAME_CONTRACT_STATUS_Signed
0,2562384,378907,0.947368,0.218211,0.100000,0.003091,0.000420,0.219614,0.006061,0.000389,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2582071,363914,1.000000,0.251395,0.033333,0.003690,0.000525,0.253516,0.006061,0.000767,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1740877,371185,0.936842,0.234699,0.333333,0.002709,0.000525,0.236054,0.000000,0.000333,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1389973,337855,0.968421,0.341002,0.166667,0.003690,0.002780,0.341400,0.006061,0.000111,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1891521,126868,1.000000,0.453842,0.333333,0.007744,0.006295,0.456822,0.006061,0.001122,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3840307,1036507,328243,0.915789,0.218181,0.033333,0.002709,0.000000,0.219614,0.000000,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3840308,1714892,347207,0.915789,0.218181,0.033333,0.002709,0.000438,0.219614,0.000000,0.000256,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3840309,1302323,215757,0.915789,0.361360,0.433333,0.120442,0.087604,0.362327,0.012121,0.000200,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3840310,1624872,430337,0.905263,0.218181,0.333333,0.002709,0.000000,0.219614,0.000000,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


## Transforming dataset into dataset with one row per client 

In [128]:
credit_card_balance_grouped = credit_card_balance.groupby(["SK_ID_CURR", "SK_ID_PREV"]).agg({
    'AMT_BALANCE': 'mean',
    'AMT_CREDIT_LIMIT_ACTUAL': 'mean',
    'AMT_DRAWINGS_CURRENT': 'mean',
    'AMT_PAYMENT_CURRENT': 'mean',
    'AMT_TOTAL_RECEIVABLE': 'mean',
    'CNT_DRAWINGS_CURRENT': 'mean',
    'CNT_INSTALMENT_MATURE_CUM': 'mean',
    'SK_DPD': 'mean',
    'SK_DPD_DEF': 'mean',
    'NAME_CONTRACT_STATUS_Active': 'max',
    'NAME_CONTRACT_STATUS_Approved': 'max',
    'NAME_CONTRACT_STATUS_Completed': 'max',
    'NAME_CONTRACT_STATUS_Demand': 'max',
    'NAME_CONTRACT_STATUS_Refused': 'max',
    'NAME_CONTRACT_STATUS_Sent proposal': 'max',
    'NAME_CONTRACT_STATUS_Signed': 'max'
})

credit_card_balance_grouped = credit_card_balance.groupby("SK_ID_CURR").agg({
    'AMT_BALANCE': 'mean',
    'AMT_CREDIT_LIMIT_ACTUAL': 'mean',
    'AMT_DRAWINGS_CURRENT': 'mean',
    'AMT_PAYMENT_CURRENT': 'mean',
    'AMT_TOTAL_RECEIVABLE': 'mean',
    'CNT_DRAWINGS_CURRENT': 'mean',
    'CNT_INSTALMENT_MATURE_CUM': 'mean',
    'SK_DPD': 'mean',
    'SK_DPD_DEF': 'mean',
    'NAME_CONTRACT_STATUS_Active': 'sum',
    'NAME_CONTRACT_STATUS_Approved': 'sum',
    'NAME_CONTRACT_STATUS_Completed': 'sum',
    'NAME_CONTRACT_STATUS_Demand': 'sum',
    'NAME_CONTRACT_STATUS_Refused': 'sum',
    'NAME_CONTRACT_STATUS_Sent proposal': 'sum',
    'NAME_CONTRACT_STATUS_Signed': 'sum'
})

# Now we have to scale the NAME_CONTRACT_STATUS columns again

columns_to_scale = ['NAME_CONTRACT_STATUS_Active','NAME_CONTRACT_STATUS_Approved',
                    'NAME_CONTRACT_STATUS_Completed', 'NAME_CONTRACT_STATUS_Demand',
                    'NAME_CONTRACT_STATUS_Refused','NAME_CONTRACT_STATUS_Sent proposal',
                    'NAME_CONTRACT_STATUS_Signed']

scaler = MinMaxScaler()

credit_card_balance_grouped[columns_to_scale] = scaler.fit_transform(credit_card_balance_grouped[columns_to_scale])

### Final dataset:

In [129]:
credit_card_balance_grouped

,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_CURRENT,AMT_PAYMENT_CURRENT,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_CURRENT,CNT_INSTALMENT_MATURE_CUM,SK_DPD,SK_DPD_DEF,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Sent proposal,NAME_CONTRACT_STATUS_Signed
SK_ID_CURR,,,,,,,,,,,,,,,,
100006,0.218181,0.200000,0.002709,0.000000,0.219614,0.000000,0.000000,0.000000,0.000000,0.033708,0.0,0.000000,0.0,0.0,0.0,0.0
100011,0.246467,0.121622,0.003769,0.001129,0.248059,0.000328,0.000286,0.000000,0.000000,0.415730,0.0,0.000000,0.0,0.0,0.0,0.0
100013,0.227609,0.097569,0.005304,0.001671,0.229073,0.001452,0.000208,0.000003,0.000003,0.539326,0.0,0.000000,0.0,0.0,0.0,0.0
100021,0.218181,0.500000,0.002709,0.000000,0.219614,0.000000,0.000000,0.000000,0.000000,0.039326,0.0,0.185185,0.0,0.0,0.0,0.0
100023,0.218181,0.100000,0.002709,0.000000,0.219614,0.000000,0.000000,0.000000,0.000000,0.044944,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456244,0.286626,0.219512,0.014413,0.007629,0.287950,0.008278,0.000151,0.000000,0.000000,0.202247,0.0,0.092593,0.0,0.0,0.0,0.0
456246,0.225001,0.100000,0.009336,0.004392,0.226354,0.015152,0.000039,0.000000,0.000000,0.044944,0.0,0.000000,0.0,0.0,0.0,0.0
456247,0.230234,0.106667,0.003646,0.001139,0.231700,0.000893,0.000294,0.000010,0.000006,0.533708,0.0,0.000000,0.0,0.0,0.0,0.0


## Installment payments



In [21]:
# Load data

installment_payments = pd.read_csv('installments_payments.csv', sep=',')

installment_payments.head(10)

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585
5,1137312,164489,1.0,12,-1384.0,-1417.0,5970.375,5970.375
6,2234264,184693,4.0,11,-349.0,-352.0,29432.295,29432.295
7,1818599,111420,2.0,4,-968.0,-994.0,17862.165,17862.165
8,2723183,112102,0.0,14,-197.0,-197.0,70.740,70.740
9,1413990,109741,1.0,4,-570.0,-609.0,14308.470,14308.470


In [22]:
# Identify duplicate rows

duplicate_rows = installment_payments[installment_payments.duplicated()]
len(duplicate_rows)

0

### Dealing with missing values

In [23]:
n_null_dict = {}

for column in installment_payments.columns:
    null_sum = installment_payments.isnull()[column].sum()
    n_null_dict[column] = null_sum

n_null = pd.DataFrame(n_null_dict, index = ["Number of missing values"])

In [24]:
n_null

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
Number of missing values,0,0,0,0,0,2905,0,2905


We only have missing values in the collumns:

- DAYS_ENTRY_PAYMENT, which says when was the installments of previous credit paid actually (relative to application date of current loan)
- AMT_PAYMENT, what the client actually paid on previous credit on this installment

We will interpret Null values as situations where the client hasn't paid the installment at all. Let's see some examples:

In [25]:
installment_payments[installment_payments['DAYS_ENTRY_PAYMENT'].isnull()]

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
3764207,1531600,103793,1.0,7,-668.0,NaN,49741.020,NaN
3764208,1947105,159974,1.0,24,-36.0,NaN,22849.515,NaN
3764209,1843773,167270,1.0,22,-20.0,NaN,48092.355,NaN
3764210,1691592,192536,1.0,5,-2561.0,NaN,7675.425,NaN
3764211,1531299,157088,0.0,11,-1847.0,NaN,67.500,NaN
...,...,...,...,...,...,...,...,...
13605396,2186857,428057,0.0,66,-1624.0,NaN,67.500,NaN
13605397,1310347,414406,0.0,47,-1539.0,NaN,67.500,NaN
13605398,1308766,402199,0.0,43,-7.0,NaN,43737.435,NaN
13605399,1062206,409297,0.0,43,-1986.0,NaN,67.500,NaN


Let's replace the null values in DAYS_ENTRY_PAYMENT with "0" so that when we calculate a new column with the delay in payment, we get the delay we have relative to application date, even though the client can take even more time to make the payment.

However, we will create a new column saying whether the client has already payed previous installment or not, to identify these cases.

For the column AMT_PAYMENT, we will replace null values with 0 (because they haven't payed yet.

**How will we analyze data**: For the cases where client has payed, check values of delay time and difference between amount of installment and amount payed. Cases where client hasn't payed yet, take only into consideration that he hasn't payed yet.

In [26]:
installment_payments['DAYS_ENTRY_PAYMENT'] = installment_payments['DAYS_ENTRY_PAYMENT'].fillna(0)
installment_payments['AMT_PAYMENT'] = installment_payments['AMT_PAYMENT'].fillna(0)

### Feature selection and new columns

DAYS_INSTALMENT columns says when the installment of previous credit was supposed to be paid, and DAYS_ENTRY_PAYMENT says when was the installments of previous credit paid actually. We'll drop those two columns and replace them with one with the difference between both. That new column will show the delay in the installment payment.

In [27]:
# Create new column DAYS_PAYMENT_DELAY

installment_payments['DAYS_INSTALMENT'] = installment_payments['DAYS_INSTALMENT'].abs()
installment_payments['DAYS_ENTRY_PAYMENT'] = installment_payments['DAYS_ENTRY_PAYMENT'].abs()

installment_payments['DAYS_PAYMENT_DELAY'] = - (installment_payments['DAYS_ENTRY_PAYMENT'] - installment_payments['DAYS_INSTALMENT'])

In [28]:
# Create new column DAYS_PAYMENT_DELAY

installment_payments['DAYS_INSTALMENT'] = installment_payments['DAYS_INSTALMENT'].abs()
installment_payments['DAYS_ENTRY_PAYMENT'] = installment_payments['DAYS_ENTRY_PAYMENT'].abs()

installment_payments['AMT_MISSING_PAYMENT'] = installment_payments['AMT_INSTALMENT'] - installment_payments['AMT_PAYMENT']

In [29]:
# Create new column INSTALLMENT_PAYED

def payed_or_not(x):
    if x == 0:
        return '0'
    else:
        return '1'

installment_payments['INSTALLMENT_PAYED'] = installment_payments['DAYS_ENTRY_PAYMENT'].apply(payed_or_not)

Since we have new columns containing all the information that matters, we can drop some of the columns we had before.

In [30]:
installment_payments.drop(['NUM_INSTALMENT_VERSION', 'DAYS_INSTALMENT',
                          'DAYS_ENTRY_PAYMENT', 'AMT_INSTALMENT',
                          "AMT_PAYMENT"], axis=1, inplace=True)

In [31]:
installment_payments

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_NUMBER,DAYS_PAYMENT_DELAY,AMT_MISSING_PAYMENT,INSTALLMENT_PAYED
0,1054186,161674,6,-7.0,0.000,1
1,1330831,151639,34,-0.0,0.000,1
2,2085231,193053,1,-0.0,0.000,1
3,2452527,199697,3,-8.0,0.000,1
4,2714724,167756,2,17.0,4.455,1
...,...,...,...,...,...,...
13605396,2186857,428057,66,1624.0,67.500,0
13605397,1310347,414406,47,1539.0,67.500,0
13605398,1308766,402199,43,7.0,43737.435,0
13605399,1062206,409297,43,1986.0,67.500,0


### Final pipeline to transform data and get new dataframe:

In [32]:
# Pipeline to transform numerical features

numerical_features = [col for col in installment_payments.columns if col != 'SK_ID_PREV' and col != 'SK_ID_CURR']

# Define column transformer to apply scaling

preprocessor = make_column_transformer(
    (MinMaxScaler(), numerical_features))

# Define the pipeline to apply the column transformer

pipeline = make_pipeline(preprocessor)

# Fit the pipeline to the original dataset 

pipeline.fit(installment_payments)

# Feature names

feature_names = numerical_features

# Select the columns to keep from the original dataframe

id_columns = ['SK_ID_PREV', 'SK_ID_CURR']

# Create a new dataframe with the transformed features

new_installment_payments = pd.DataFrame(pipeline.transform(installment_payments), columns=feature_names)

# Concatenate the transformed features dataframe with the original id columns

installment_payments = pd.concat([installment_payments[id_columns], new_installment_payments], axis=1)

In [33]:
installment_payments

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_NUMBER,DAYS_PAYMENT_DELAY,AMT_MISSING_PAYMENT,INSTALLMENT_PAYED
0,1054186,161674,0.018116,0.521896,0.520391,1.0
1,1330831,151639,0.119565,0.523044,0.520391,1.0
2,2085231,193053,0.000000,0.523044,0.520391,1.0
3,2452527,199697,0.007246,0.521732,0.520391,1.0
4,2714724,167756,0.003623,0.525832,0.520392,1.0
...,...,...,...,...,...,...
13605396,2186857,428057,0.235507,0.789405,0.520405,0.0
13605397,1310347,414406,0.166667,0.775463,0.520405,0.0
13605398,1308766,402199,0.152174,0.524192,0.529043,0.0
13605399,1062206,409297,0.152174,0.848778,0.520405,0.0
